#### Import dos pacotes e set de configurações

In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

pd.set_option('display.max_rows', 500)

#### Import dos dados

In [2]:
acid_df = pd.read_csv('acidentes-apriori.csv', sep=';')

In [3]:
acid_df.head()

,dia_semana,uf,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,condicao_metereologica,tracado_via
0,terca-feira,SP,Falta de Atencao Conducao,Colisao com objeto estatico,Com Vitimas Feridas,Plena Noite,Ceu Claro,Curva
1,terca-feira,PR,Falta de Atencao Conducao,Colisao traseira,Com Vitimas Feridas,Plena Noite,Nublado,Reta
2,terca-feira,SC,Animais na Pista,Colisao com objeto estatico,Com Vitimas Feridas,Plena Noite,Ceu Claro,Reta
3,terca-feira,CE,Ingestao de Substancias Psicoativas,Colisao com objeto estatico,Com Vitimas Feridas,Plena Noite,Nublado,Viaduto
4,terca-feira,MG,Falta de Atencao Conducao,Colisao transversal,Com Vitimas Feridas,Plena Noite,Nublado,Intersecao de vias


In [4]:
# formato dos dados
acid_df.shape

(20882, 8)

Vemos que são cerca de 20k registros, com 8 colunas

In [5]:
# verificando se há alguma resposta nula
acid_df.isna().sum()

dia_semana                0
uf                        0
causa_acidente            0
tipo_acidente             0
classificacao_acidente    0
fase_dia                  0
condicao_metereologica    0
tracado_via               0
dtype: int64

In [6]:
# vamos entender as proporções de cada resposta no df
for col in acid_df.columns:
    print('\n\n ---------{}---------'.format(col))
    print(acid_df[col].value_counts(normalize=True))



 ---------dia_semana---------
sabado           0.167177
domingo          0.162053
sexta-feira      0.149555
segunda-feira    0.134805
terca-feira      0.132698
quinta-feira     0.129442
quarta-feira     0.124270
Name: dia_semana, dtype: float64


 ---------uf---------
MG    0.130112
SC    0.126233
PR    0.115458
RJ    0.069342
RS    0.066517
SP    0.065942
BA    0.050139
GO    0.049852
ES    0.041711
PE    0.041088
MT    0.031558
CE    0.025764
PB    0.024710
RO    0.021214
MS    0.020640
PI    0.019874
RN    0.019778
MA    0.016378
DF    0.014366
PA    0.011876
SE    0.010200
AL    0.010104
TO    0.007710
RR    0.003352
AC    0.002873
AP    0.001963
AM    0.001245
Name: uf, dtype: float64


 ---------causa_acidente---------
Falta de Atencao Conducao                                                           0.363136
Desobediencia nas normas de transito pelo condutor                                  0.114501
Velocidade Incompativel                                                      

#### Transformando os dados em lista de listas

In [7]:
# aplicando a transformação para a lista de listas

acid_lst = []
for i in range(0, acid_df.shape[0]):
    acid_cols = []
    for j in range(0, acid_df.shape[1]):        
        acid_cols.append(str(acid_df.values[i,j]))
    acid_lst.append(acid_cols)

In [8]:
# visualizando as 5 primeiras entradas
acid_lst[:5]

[['terca-feira',
  'SP',
  'Falta de Atencao Conducao',
  'Colisao com objeto estatico',
  'Com Vitimas Feridas',
  'Plena Noite',
  'Ceu Claro',
  'Curva'],
 ['terca-feira',
  'PR',
  'Falta de Atencao Conducao',
  'Colisao traseira',
  'Com Vitimas Feridas',
  'Plena Noite',
  'Nublado',
  'Reta'],
 ['terca-feira',
  'SC',
  'Animais na Pista',
  'Colisao com objeto estatico',
  'Com Vitimas Feridas',
  'Plena Noite',
  'Ceu Claro',
  'Reta'],
 ['terca-feira',
  'CE',
  'Ingestao de Substancias Psicoativas',
  'Colisao com objeto estatico',
  'Com Vitimas Feridas',
  'Plena Noite',
  'Nublado',
  'Viaduto'],
 ['terca-feira',
  'MG',
  'Falta de Atencao Conducao',
  'Colisao transversal',
  'Com Vitimas Feridas',
  'Plena Noite',
  'Nublado',
  'Intersecao de vias']]

#### Aplicação do Transaction Encoder

In [9]:
te = TransactionEncoder()

# Coloca em memória as trasações e interpreta a quantidade de colunas 
# que serão geradas durante o processamento
te.fit(acid_lst)

TransactionEncoder()

In [10]:
# O objeto TransactionEncoder faz a conversão das transações em uma matriz binária,
# onde cada linha da matriz representa uma transação
matriz_transacoes = te.transform(acid_lst)

# Cria um dataframe auxiliar com a matriz binária (passo te.transform(transacoes)) 
# de transações e as colunas obtidas (passo te.fit(transacoes))
dfAuxiliar = pd.DataFrame(matriz_transacoes, columns=te.columns_)

dfAuxiliar.head()

,AC,AL,AM,AP,Agressao Externa,Amanhecer,Animais na Pista,Anoitecer,Atropelamento de Animal,Atropelamento de Pedestre,...,Velocidade Incompativel,Vento,Viaduto,domingo,quarta-feira,quinta-feira,sabado,segunda-feira,sexta-feira,terca-feira
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


Neste ponto vale destacar que um suporte muito baixo dá uma quantidade muito alta de combinações, e os *insights* obtidos não serão de grande valor, pois representam poucos casos do total. Assim, optaremos por um **suporte mínimo de 5%**, que mantém os principais tipos de colisão em análise.

In [11]:
# Obtêm os itemsets mais frequentes com um suporte mínimo igual a 5%
# O paramêtro use_colnames significa que vamos usar os nomes das colunas do DataFrame 
# dfAuxiliar para construir as regras de Associação

itemsets_freq = apriori(dfAuxiliar, min_support=0.05, use_colnames=True)
print(itemsets_freq.head())

    support                       itemsets
0  0.053683                    (Anoitecer)
1  0.050139                           (BA)
2  0.539556                    (Ceu Claro)
3  0.132698                        (Chuva)
4  0.064122  (Colisao com objeto estatico)


#### Obtenção das regras de Associação

In [40]:
# Obtém as regras de associação a partir dos itemsets mais frequentes.
# Vamos definir uma confiança mínima de 50% para análise, de forma a 
# formar análises fortes
regras = association_rules(itemsets_freq, metric="confidence", min_threshold=0.5)

# Ordena as Regras por confiança
regrasOrdenadas = regras.sort_values('confidence' , ascending=False)

In [41]:
regrasOrdenadas.shape

(315, 9)

Temos cerca de 315 regras possíveis a analisar

In [42]:
regrasOrdenadas.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
79,(Sol),(Pleno dia),0.079734,0.573221,0.078345,0.982583,1.714143,0.032640,24.503004
221,"(Com Vitimas Feridas, Sol)",(Pleno dia),0.065319,0.573221,0.064170,0.982405,1.713832,0.026728,24.255276
170,"(Pleno dia, Colisao transversal)",(Com Vitimas Feridas),0.084092,0.760895,0.072982,0.867882,1.140607,0.008997,1.809781
168,"(Falta de Atencao Conducao, Colisao transversal)",(Com Vitimas Feridas),0.065703,0.760895,0.056891,0.865889,1.137988,0.006898,1.782895
215,"(Pleno dia, SC)",(Com Vitimas Feridas),0.074705,0.760895,0.064074,0.857692,1.127216,0.007231,1.680200


## Análise das Regras de Associação

##### Vamos analisar os casos com **vítimas fatais**:

In [58]:
termo = 'Com Vitimas Fatais'
# criando lista com índices de interesse
filtro = []
for index, row in regrasOrdenadas.iterrows():
    for ent in row['antecedents']:
        if ent == termo:
            filtro.append(index)
            break

regrasOrdenadas.loc[regrasOrdenadas.index.isin(filtro), : ][['antecedents','consequents','confidence', 'support']]

,antecedents,consequents,confidence,support


In [57]:
termo = 'Com Vitimas Fatais'
# criando lista com índices de interesse
filtro = []
for index, row in regrasOrdenadas.iterrows():
    for ent in row['consequents']:
        if ent == termo:
            filtro.append(index)
            break

regrasOrdenadas.loc[regrasOrdenadas.index.isin(filtro), : ][['antecedents','consequents','confidence', 'support']]

,antecedents,consequents,confidence,support


 - Vemos que não há regras fortes associadas aos casos de vítimas fatais

##### Agora analisando os **casos sem vítimas**:

In [56]:
termo = 'Sem Vitimas'
# criando lista com índices de interesse
filtro = []
for index, row in regrasOrdenadas.iterrows():
    for ent in row['antecedents']:
        if ent == termo:
            filtro.append(index)
            break

regrasOrdenadas.loc[regrasOrdenadas.index.isin(filtro), : ][['antecedents','consequents','confidence', 'support']]

,antecedents,consequents,confidence,support
148,"(Ceu Claro, Sem Vitimas)",(Reta),0.645812,0.059812
89,(Sem Vitimas),(Reta),0.602522,0.105258
260,"(Pleno dia, Sem Vitimas)",(Reta),0.587597,0.054449
149,"(Sem Vitimas, Reta)",(Ceu Claro),0.568244,0.059812
78,(Sem Vitimas),(Pleno dia),0.530428,0.092664
14,(Sem Vitimas),(Ceu Claro),0.530154,0.092616
261,"(Sem Vitimas, Reta)",(Pleno dia),0.517288,0.054449


- Vemos que, em 60% dos acidentes sem vítimas, o acidente ocorreu em retas. Também, 53% ocorreram em pleno dia e 53% com céu claro

##### Analisando os casos com **vítimas feridas**:

In [53]:
termo = 'Com Vitimas Feridas'
# criando lista com índices de interesse
filtro = []
for index, row in regrasOrdenadas.iterrows():
    for ent in row['consequents']:
        if ent == termo:
            filtro.append(index)
            break

regrasOrdenadas.loc[regrasOrdenadas.index.isin(filtro), : ][['antecedents','consequents','confidence', 'support']]

,antecedents,consequents,confidence,support
170,"(Pleno dia, Colisao transversal)",(Com Vitimas Feridas),0.867882,0.072982
168,"(Falta de Atencao Conducao, Colisao transversal)",(Com Vitimas Feridas),0.865889,0.056891
215,"(Pleno dia, SC)",(Com Vitimas Feridas),0.857692,0.064074
28,(Colisao transversal),(Com Vitimas Feridas),0.854784,0.109089
99,"(Ceu Claro, Colisao transversal)",(Com Vitimas Feridas),0.847348,0.062733
173,"(Colisao transversal, Reta)",(Com Vitimas Feridas),0.847034,0.056747
187,"(Pleno dia, Desobediencia nas normas de transi...",(Com Vitimas Feridas),0.839286,0.058519
158,"(Falta de Atencao Conducao, Colisao lateral)",(Com Vitimas Feridas),0.831595,0.053443
309,"(Pleno dia, Falta de Atencao Conducao, Reta, C...",(Com Vitimas Feridas),0.831299,0.071736
304,"(Pleno dia, Reta, Falta de Atencao Conducao)",(Com Vitimas Feridas),0.830846,0.119960


- Vemos que o tipo de acidente mais grave é o ocorrido em pleno dia, por colisão transversal, que implica em Vítimas Feridas em 87% dos casos!
- Também, os acidentes em Pleno dia em SC implicam em vítimas feridas em 87% dos casos. 
- Analisando somente os estados, temos um indicativo de que realmente SC tende a ter acidentes com vítimas feridas maior que outros estados. SC apresenta acidentes com vítimas feridas em 80,9% dos casos. Já o PR apresenta acidentes com vítimas feridas em 75% dos casos. 
- Terça-feira tem acidentes com vítimas feridas em 78% dos casos, enquanto aos sábados e domingos este percentual é menor, com cerca de 73% dos casos.

##### Vamos agora analisar os **casos de ingestão de álcool**:

In [46]:
termo = 'Ingestao de Alcool'
# criando lista com índices de interesse
filtro = []
for index, row in regrasOrdenadas.iterrows():
    for ent in row['antecedents']:
        if ent == termo:
            filtro.append(index)
            break

regrasOrdenadas.loc[regrasOrdenadas.index.isin(filtro), : ][['antecedents','consequents','confidence', 'support']]

,antecedents,consequents,confidence,support


- Não há nenhuma regra forte associada ao consumo de álcool

##### Analisando agora os casos de **Velocidade Incompatível** com a via:

In [47]:
termo = 'Velocidade Incompativel'
# criando lista com índices de interesse
filtro = []
for index, row in regrasOrdenadas.iterrows():
    for ent in row['antecedents']:
        if ent == termo:
            filtro.append(index)
            break

regrasOrdenadas.loc[regrasOrdenadas.index.isin(filtro), : ][['antecedents','consequents','confidence', 'support']]

,antecedents,consequents,confidence,support
54,(Velocidade Incompativel),(Com Vitimas Feridas),0.721270,0.068528
80,(Velocidade Incompativel),(Pleno dia),0.608871,0.057849


- 72% dos acidentes por Velocidade Incompatível tiveram vítimas feridas
- 61% dos acidentes por Velocidade Incompatível ocorreram em pleno dia

##### Analisando agora os casos de **Falta de Atenção na Condução**:

In [48]:
termo = 'Falta de Atencao Conducao'
# criando lista com índices de interesse
filtro = []
for index, row in regrasOrdenadas.iterrows():
    for ent in row['antecedents']:
        if ent == termo:
            filtro.append(index)
            break

regrasOrdenadas.loc[regrasOrdenadas.index.isin(filtro), : ][['antecedents','consequents','confidence', 'support']]

,antecedents,consequents,confidence,support
168,"(Falta de Atencao Conducao, Colisao transversal)",(Com Vitimas Feridas),0.865889,0.056891
158,"(Falta de Atencao Conducao, Colisao lateral)",(Com Vitimas Feridas),0.831595,0.053443
309,"(Pleno dia, Falta de Atencao Conducao, Reta, C...",(Com Vitimas Feridas),0.831299,0.071736
304,"(Pleno dia, Reta, Falta de Atencao Conducao)",(Com Vitimas Feridas),0.830846,0.119960
276,"(Pleno dia, Falta de Atencao Conducao, Ceu Claro)",(Com Vitimas Feridas),0.828264,0.106934
194,"(Pleno dia, Falta de Atencao Conducao)",(Com Vitimas Feridas),0.827762,0.186189
280,"(Falta de Atencao Conducao, Reta, Ceu Claro)",(Com Vitimas Feridas),0.819425,0.114740
197,"(Falta de Atencao Conducao, Reta)",(Com Vitimas Feridas),0.817427,0.193181
192,"(Falta de Atencao Conducao, Nublado)",(Com Vitimas Feridas),0.817131,0.057562
112,"(Falta de Atencao Conducao, Ceu Claro)",(Com Vitimas Feridas),0.816364,0.168183


- Os casos de falta de atenção na condução implicam em acidentes com vítimas feridas em 81% dos casos!
- Quando comparados aos casos de velocidade incompatível, nota-se que este caso é mais grave, pois o % calculado anteriormente é 72% dos casos, sugerindo que este tipo de acidente é mais perigoso
- Quando analisados os casos de falta de atenção com colisão transversal, este caso é ainda mais grave, com 86,6% dos casos com acidentes com vítimas feridas. Análise similar pode-se fazer com as colisões laterais, com taxa de acidentes com vítimas feridas de 83% dos casos

##### Analisando agora os casos de **Desobediência das Normas de Trânsito**:

In [36]:
termo = 'Desobediencia nas normas de transito pelo condutor'
# criando lista com índices de interesse
filtro = []
for index, row in regrasOrdenadas.iterrows():
    for ent in row['antecedents']:
        if ent == termo:
            filtro.append(index)
            break

regrasOrdenadas.loc[regrasOrdenadas.index.isin(filtro), : ][['antecedents','consequents','confidence', 'support']]

,antecedents,consequents,confidence,support
270,"(Pleno dia, Desobediencia nas normas de transi...",(Com Vitimas Feridas),0.839286,0.058519
143,"(Ceu Claro, Desobediencia nas normas de transi...",(Com Vitimas Feridas),0.817601,0.055167
274,(Desobediencia nas normas de transito pelo con...,(Com Vitimas Feridas),0.816460,0.053683
43,(Desobediencia nas normas de transito pelo con...,(Com Vitimas Feridas),0.813049,0.093095
271,(Desobediencia nas normas de transito pelo con...,(Pleno dia),0.628601,0.058519
72,(Desobediencia nas normas de transito pelo con...,(Pleno dia),0.608950,0.069725
144,(Desobediencia nas normas de transito pelo con...,(Ceu Claro),0.592593,0.055167
6,(Desobediencia nas normas de transito pelo con...,(Ceu Claro),0.589293,0.067474
273,(Desobediencia nas normas de transito pelo con...,(Reta),0.576646,0.053683
73,(Desobediencia nas normas de transito pelo con...,(Reta),0.574237,0.065750


- Os casos de Desobediência às Leis em Pleno Dia também são graves, uma vez que implicam em vítimas feridas em ~84% dos casos.

### Conclusões

Alguns *insights* importantes podem ser obtidos a partir dos dados analisados, como:
- Em 60% dos acidentes sem vítimas, o acidente ocorreu em retas. Também, 53% ocorreram em pleno dia e 53% com céu claro. Nenhuma destas regras, entretanto, pode ser obtida nos casos de acidentes com vítimas fatais

Sobre a gravidade dos acidentes:
- O tipo de acidente mais grave é o ocorrido em pleno dia, por colisão transversal, que implica em Vítimas Feridas em 87% dos casos!
- Os acidentes em Pleno dia em SC implicam em vítimas feridas em 87% dos casos!
- Comparando somente os estados, temos um indicativo de que realmente SC tende a ter acidentes com vítimas feridas maior que outros estados. SC apresenta acidentes com vítimas feridas em 80,9% dos casos. Já o PR apresenta acidentes com vítimas feridas em 75% dos casos. Aqui vale ressaltar que não temos todos os estados inclusos na análise, por não atingirem o suporte ou a confiança mínima. O que se deseja aqui é mostrar que SC aparenta implicar em acidentes mais graves que outros estados.
- Terça-feira tem acidentes com vítimas feridas em 78% dos casos, enquanto aos sábados e domingos este percentual é menor, com cerca de 73% dos casos.

Sobre a causa dos acidentes:

- Nao pode-se obter *insights* fortes sobre acidentes ocorridos por ingestão de álcool
- Acidentes por Falta de Atenção do Condutor e por Desobediência das Normas de Trânsito são mais graves que acidentes por Velocidade Incompatíveis com a Via, pois implicam em maior percentual de acidentes com vítimas feridas.